<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean , median
import matplotlib.pyplot as plt

# Baseline 

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps[nb.scraps['TabNet MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["TabNet(LR=0.02)"])
baseline_data = np.array(baseLine_data)
tabnet02 = median(baseline_data[:,2])
tabnet1 = median(baseline_data[:,3])

SyntaxError: invalid syntax (<ipython-input-3-4d4def7fb8d4>, line 5)

# GAN Model

In [ ]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEDIAN:")
print(df.median(axis = 0))
gan_data = np.array(gan_data)
gan_median = median(gan_data[:,1])
print(gan_median)

# ABC_GAN Analysis

## ABC Pre-generator - TabNet 

In [ ]:
book = sb.read_notebooks("./ABC_GAN_TabNet")
paramVal = [[1,1],[1,0.1],[1,0.01],[1,0],[0.1,1],[0.1,0.1],[0.1,0.01],[0.1,0],[0.01,1],[0.01,0.1],[0.01,0.01],[0.01,0]]
abc_mae = [[] for i in range(12)]
abc_mae_skip = [[] for i in range(12)]
abc_mae_mean = [[] for i in range(12)]
abc_mae_skip_mean = [[] for i in range(12)]
abc_weights = [[] for i in range(12)]
prior_model = [[] for i in range(12)]
abc_pre_generator = [[] for i in range(12)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    paramBias = float(nb.papermill_dataframe.iloc[1]['value'])
    #Divide data according to parameters 
    for i in range(9):
        if paramVar == paramVal[i][0] and paramBias == paramVal[i][1]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [ ]:
data = [[] for i in range(12)]
for i in range(12):
    for j in range(len(abc_weights[i])):
        data[i].append([paramVal[i][0], paramVal[i][1],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    df = pd.DataFrame(data[i], columns = ['Variance','Bias','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.median(axis=0))
    print("-------------------------------------------------------------")

In [ ]:
# Display Catboost Summary Tables 
data = np.array(data)
tabnetData = []
for i in range(12): 
    tabnetData.append([paramVal[i][0], paramVal[i][1],catboost,median(data[i][:,3]),median(data[i][:,5]),median(data[i][:,6]),median(data[i][:,4])])
df = pd.DataFrame(tabnetData, columns = ['Variance','Bias','TabNet','Prior Model MAE','mGAN','skipGAN','Skip Node weight'])
display(df.round(5))